## Slice mesh to squares

In [1]:
import point_cloud_utils as pcu
import os
import glob
from tqdm import tqdm
import numpy as np
import pandas as pd
import trimesh

from trimesh import transform_points

In [2]:
class Config:
    apply_pose = True
    min_range = 2.0
    max_range = 25.0


class KITTIOdometryDataset:
    def __init__(self, kitti_root_dir: str, sequence: int):
        """Simple KITTI DataLoader to provide a ready-to-run example.

        Heavily inspired in PyLidar SLAM
        """
        # Config stuff
        self.sequence = str(int(sequence)).zfill(2)
        self.config = Config()
        self.kitti_sequence_dir = os.path.join(kitti_root_dir, "sequences", self.sequence)
        self.velodyne_dir = os.path.join(self.kitti_sequence_dir, "velodyne/")

        # Read stuff
        self.calibration = self.read_calib_file(os.path.join(self.kitti_sequence_dir, "calib.txt"))
        self.poses = self.load_poses(os.path.join(kitti_root_dir, f"poses/{self.sequence}.txt"))
        self.scan_files = sorted(glob.glob(self.velodyne_dir + "*.bin"))

    def __getitem__(self, idx):
        if idx < 0:
            idx = len(self.scan_files) + idx
        elif idx > len(self.scan_files):
            idx -= len(self.scan_files)
        return self.scans(idx), self.poses[idx]

    def get_pose(self, idx):
        return self.poses[idx]

    def __len__(self):
        return len(self.scan_files)

    def scans(self, idx):
        return self.read_point_cloud(idx, self.scan_files[idx], self.config)

    def read_point_cloud(self, idx: int, scan_file: str, config: Config):
        points = np.fromfile(scan_file, dtype=np.float32).reshape((-1, 4))[:, :-1]
        points = points[np.linalg.norm(points, axis=1) <= config.max_range]
        points = points[np.linalg.norm(points, axis=1) >= config.min_range]
        points = transform_points(points, self.poses[idx], translate=True) if config.apply_pose else points
        return points

    def load_poses(self, poses_file):
        def _lidar_pose_gt(poses_gt):
            _tr = self.calibration["Tr"].reshape(3, 4)
            tr = np.eye(4, dtype=np.float64)
            tr[:3, :4] = _tr
            left = np.einsum("...ij,...jk->...ik", np.linalg.inv(tr), poses_gt)
            right = np.einsum("...ij,...jk->...ik", left, tr)
            return right

        poses = pd.read_csv(poses_file, sep=" ", header=None).values
        n = poses.shape[0]
        poses = np.concatenate(
            (poses, np.zeros((n, 3), dtype=np.float32), np.ones((n, 1), dtype=np.float32)), axis=1
        )
        poses = poses.reshape((n, 4, 4))  # [N, 4, 4]
        return _lidar_pose_gt(poses)

    @staticmethod
    def read_calib_file(file_path: str) -> dict:
        calib_dict = {}
        with open(file_path, "r") as calib_file:
            for line in calib_file.readlines():
                tokens = line.split(" ")
                if tokens[0] == "calib_time:":
                    continue
                # Only read with float data
                if len(tokens) > 0:
                    values = [float(token) for token in tokens[1:]]
                    values = np.array(values, dtype=np.float32)
                    # The format in KITTI's file is <key>: <f1> <f2> <f3> ...\n -> Remove the ':'
                    key = tokens[0][:-1]
                    calib_dict[key] = values
        return calib_dict

In [3]:
dataset = KITTIOdometryDataset(kitti_root_dir="/home/kowalski/data/kitti", sequence=0)

### Classic

In [4]:
MESH_PATH = "/home/kowalski/data/maps_for_comparison/vdb_mesh.ply"
MAP_NAME = 'vdb_map'
OUT_DIR = f"/home/kowalski/data/sliced_maps/{MAP_NAME}"
SAMPLE_FREQ = 40

### For semantic kimera

In [5]:
from semantic_kitti_config import labels

In [6]:
MESHES_DIR = "/home/kowalski/data/maps_for_comparison/semantic_kimera/"
MAP_NAME = 'kimera_true_semantic'
OUT_DIR = f"/home/kowalski/data/sliced_maps/{MAP_NAME}"
SAMPLE_FREQ = 40

In [7]:
MESHES = sorted([x.split('.')[0] for x in os.listdir(MESHES_DIR)])[5:6]

In [9]:
if not OUT_DIR:
    os.makedirs(OUT_DIR)

In [10]:
for mesh in MESHES: 
  if not os.path.exists(os.path.join(OUT_DIR, mesh)):
    os.makedirs(os.path.join(OUT_DIR, mesh))

In [ ]:
for mesh in MESHES:
    box = trimesh.creation.box(extents=(50, 50, 50))
    map = trimesh.exchange.load.load(os.path.join(MESHES_DIR, f"{mesh}.ply"))
    for idx in tqdm(range(len(dataset))[::SAMPLE_FREQ]):
        if idx <4290:
            continue
        pose = dataset.get_pose(idx)
        pose_inv = np.linalg.inv(pose)
        box.apply_transform(pose)
        result = map.slice_plane(box.facets_origin, -box.facets_normal)
        box.apply_transform(pose_inv)
        trimesh.exchange.export.export_mesh(result, os.path.join(OUT_DIR,mesh,f"slice_{str(idx)}.ply"), file_type="ply")
        #trimesh.exchange.export.export_mesh(box, os.path.join(OUT_DIR,mesh,f"box_{str(idx)}.ply"), file_type="ply")


## Sample point clouds from lidar scans

In [7]:
dataset = KITTIOdometryDataset(kitti_root_dir="/home/kowalski/data/kitti", sequence=0)

In [9]:
MAP_NAME = "kimera_gt"
OUT_DIR = f"/home/kowalski/data/sample_pcls/{MAP_NAME}"
SAMPLE_NUM = int(5e6)
SAMPLE_RANGE = 25
SAMPLE_FREQ = 40

if not os.path.exists(OUT_DIR):
  os.makedirs(OUT_DIR)

In [10]:
def get_points_in_range(points, pose, range):
    inv_pose = np.linalg.inv(pose)
    points_t = transform_points(points, inv_pose, translate=True)
    
    # cut points inside specidic range
    x_min, x_max = -range, range
    y_min, y_max = -range, range
    z_min, z_max = -range, range
    ll = np.array([x_min, y_min, z_min])
    ur = np.array([x_max, y_max, z_max])

    inidx = np.all(np.logical_and(ll <= points_t, points_t <= ur), axis=1)

    return points[inidx]
    

In [ ]:
for idx (scan, pose) in tqdm(enumerate(dataset)):
    if idx // SAMPLE_FREQ:
        continue
    merged_pcl = scan
    for _i in range(len(dataset)):
        if _i == idx:
            continue
        p = dataset.get_pose(_i)
        if np.linalg.norm(p[:3,3] - pose[:3, 3]) < SAMPLE_RANGE:
            s, p = dataset[_i]
            merged_pcl = np.concatenate((merged_pcl, s), axis=0)
        #print(len(merged_pcl))
    points_inside = get_points_in_range(merged_pcl, pose, SAMPLE_RANGE)
    replace = len(points_inside) < SAMPLE_NUM
    if replace:
        print(f"Replace!: {len(points_inside)}" )
    sampled_points = points_inside[np.random.choice(range(len(points_inside)), SAMPLE_NUM, replace=replace)]
    sampled_points = trimesh.PointCloud(sampled_points)
    sampled_points.export(os.path.join(OUT_DIR, f"sampled_pcl_{idx}.ply"), file_type="ply")

## Sample point clouds meshes

In [ ]:
MAP_NAME = "vdb_map"
MESH_PATH = f"/home/kowalski/data/sliced_maps/{MAP_NAME}/"
SAMPLED_PATH = f"/home/kowalski/data/sample_pcls/{MAP_NAME}/"
GT = f"/home/kowalski/data/sample_pcls/gt/"
SAMPLE_NUM = int(5e6)
OUT_DIR = f"/home/kowalski/data/sample_pcls/{MAP_NAME}"

meshes = sorted(os.listdir(MESH_PATH), key=lambda s: int(s.split('_')[-1].split('.')[0]))
pcl_clouds = sorted(os.listdir(GT), key=lambda s: int(s.split('_')[-1].split('.')[0]))
sampled_clouds = sorted(os.listdir(GT), key=lambda s: int(s.split('_')[-1].split('.')[0]))

if not os.path.exists(OUT_DIR):
  os.makedirs(OUT_DIR)

In [13]:
import point_cloud_utils as pcu
import open3d as o3d

In [ ]:
for sample_p, pcl_p in tqdm(zip(sampled_clouds, pcl_clouds)):

    idx = int(pcl_p.split('_')[-1].split('.')[0])
    pcd = o3d.io.read_point_cloud(os.path.join(GT, pcl_p))
    p = np.asarray(pcd.points)

    pcd = o3d.io.read_point_cloud(os.path.join(SAMPLED_PATH, sample_p))
    v = np.asarray(pcd.points)

    idx = np.random.choice(range(len(p)), int(0.2* v.shape[0]))

    # Use the indices to get the sample positions and normals
    p_sampled = p[idx]
    v_sampled = v[idx]
    
    base = trimesh.PointCloud(v_sampled, colors=colors_b)
    gt = trimesh.PointCloud(p_sampled, colors=colors_g)
    
    d, fi, bc = pcu.closest_points_on_mesh(p, v, f)
    closest_points = pcu.interpolate_barycentric_coords(f, fi, bc, v)
    pcl = trimesh.PointCloud(closest_points)
    idx = int(mesh_p.split('_')[-1].split('.')[0])
    pcl.export(os.path.join(OUT_DIR, f"sampled_pcl_{idx}.ply"), file_type="ply")

## Metrics!

In [23]:
import os
import csv
import open3d as o3d
import numpy as np
from metrics import CloudMetrics
from tqdm import tqdm

In [ ]:
MAP_NAME = "poisson_map"
SAMPLES_PATH = f"/home/kowalski/data/sample_pcls/{MAP_NAME}"
GT = f"/home/kowalski/data/sample_pcls/gt/"
OUT_DIR = f"./metrics/{MAP_NAME}"

samples_mesh = sorted(os.listdir(SAMPLES_PATH), key=lambda s: int(s.split('_')[-1].split('.')[0]))[:100]
samples_gt = sorted(os.listdir(GT), key=lambda s: int(s.split('_')[-1].split('.')[0]))[:100]

if not os.path.exists(OUT_DIR):
  os.makedirs(OUT_DIR)

In [ ]:
metrics_log = {"Hdf_fw":[], "Hdf_bw":[], "CD":[], "precision":[], "recall": [], "f-score": []}
for mesh_p, gt_p in tqdm(zip(samples_mesh, samples_gt)):
    mesh_sample = o3d.io.read_point_cloud(os.path.join(SAMPLES_PATH, mesh_p))
    gt_sample = o3d.io.read_point_cloud(os.path.join(GT, gt_p))
    metrics = CloudMetrics(mesh_sample, gt_sample)
    hausdorff_forward, hausdorff_backward = metrics.hausdorff_distance()
    chamfer_distance = metrics.chamfer_distance()
    P, R, F = metrics.pr_f_score(0.03)
    metrics_log["Hdf_fw"].append(hausdorff_forward)
    metrics_log["Hdf_bw"].append(hausdorff_backward)
    metrics_log["CD"].append(chamfer_distance)
    metrics_log["precision"].append(P)
    metrics_log["recall"].append(R)
    metrics_log["f-score"].append(F)

In [ ]:
print(f"{MAP_NAME} Evaluation:")
print("Forward Hausdorff distance: ", np.median(np.array(metrics_log["Hdf_fw"])).round(5))
print("Bakward Hausdorff distance: ",  np.median(np.array(metrics_log["Hdf_bw"])).round(5))
print("          Chamfer distance: ",  np.median(np.array(metrics_log["CD"])).round(5))
print("                 Precision: ", np.round( np.median(np.array(metrics_log["precision"])), 5))
print("                    Recall: ", np.round( np.median(np.array(metrics_log["recall"])), 5))
print("                   F-score: ", np.round( np.median(np.array(metrics_log["f-score"])), 5))

In [8]:
with open(os.path.join(OUT_DIR, "results.csv"), 'w',  newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['id',
                        'Forward Hausdorff distance',
                        "Bakward Hausdorff distance",
                        "Chamfer distance",
                        "Precision",
                        "Recall",
                        "F-score"
                        ])
    for idx, sample in  enumerate(samples_mesh):
        csvwriter.writerow([str(int(sample.split('_')[-1].split('.')[0])),
                            str(metrics_log["Hdf_fw"][idx]),
                            str(metrics_log["Hdf_bw"][idx]),
                            str(metrics_log["CD"][idx]),
                            str(metrics_log["precision"][idx]),
                            str(metrics_log["recall"][idx]),
                            str(metrics_log["f-score"][idx]),
                           ])

In [27]:
with open(os.path.join(OUT_DIR, 'metric.txt'), 'w+') as f:
    f.write(f"{MAP_NAME} Evaluation:\n")
    f.write(f"Forward Hausdorff distance: {np.array(metrics_log['Hdf_fw']).mean().round(5)}\n")
    f.write(f"Bakward Hausdorff distance: {np.array(metrics_log['Hdf_bw']).mean().round(5)}\n")
    f.write(f"          Chamfer distance: {np.array(metrics_log['CD']).mean().round(5)}\n")
    f.write(f"                 Precision: {np.round( np.array(metrics_log['precision']).mean(), 5)}\n")
    f.write(f"                    Recall: {np.round( np.array(metrics_log['recall']).mean(), 5)}\n")
    f.write(f"                   F-score: {np.round( np.array(metrics_log['f-score']).mean(), 5)}\n")